In [2]:
import emcee
import os
import rebound
import numpy as np
import pandas as pd
from scipy import stats
import astropy.constants as ast
import matplotlib.pyplot as plt

### Data Import

In [3]:
const ={
    'G' :  6.67430e10-11, #SI
    'R_b' : 0.0677, #AU
    'R_c' : 0.1189 , #AU
    'R_d' : 0.1662, #AU
    'R_e' : 0.2138, #AU
    'R_f' : 0.2535, #AU
    'P_b' : 5.7, # days
    'P_c' : 13.2, # days
    'P_d' : 21.8, # days
    'P_e' : 31.8, # days
    'P_f' : 41,# days
    'M_b' : 3.68, #Earth
    'M_c' : 0.39, #Earth
    'M_d': 3.91, #Earth
    'M_e' : 5.57, #Earth
    'M_f': 9.6, #Earth
    'AU' : 1.496e11, #m
    'M_star' : 1.29, # solar mass
    'M_sun' : 1.988e30, # kg
    'day_to_sec' : 86400,
    'M_earth' : 5972e24, #kg
    'M_earth_sun' : 3e-6 # solar masses
}

In [4]:
data1 = np.genfromtxt('koi707_timing/koi0707.01.tt')
data2 = np.genfromtxt('koi707_timing/koi0707.02.tt')
data3 = np.genfromtxt('koi707_timing/koi0707.03.tt')
data4 = np.genfromtxt('koi707_timing/koi0707.04.tt')
data5 = np.genfromtxt('koi707_timing/koi0707.05.tt')

data = [data1, data2, data3, data4, data5] # All data together
x = np.zeros_like(data1[:,0]) # Setting up dummy variable for later

y_array = [] 
y_err_array = []

for i in range(len(data)):
    
    inter = data[i]
    
    y_array.append(inter[:,0]+inter[:,1]) # Adding the first 2 columns of data
    y_err_array.append(inter[:,2]) # Creating uncertainty array

    
## Planet c (index 1) has the smallest initial time
final_y = []
for i in (y_array):
    final_y.append(i - y_array[1][0]) # Resetting time to get c at s t=0 starting time

In [5]:
def fCalc(b4T0, P, e):
    '''
    To approximate the starting positions. First, we find the mean ananomaly, taking T0 as periapsis.
    Then, we approximate the eccentric ananomaly as the mean ananomaly as e is small. We then directly
    derive our true ananomaly from our eccentric ananomaly, given as the alternate form from
    https://en.wikipedia.org/wiki/True_anomaly
    '''
    M = (P - b4T0)/P # mean ananomaly approximately eccentric ananomaly when e is small.
    # The calculation to derive the true 
    beta = e/(1+np.sqrt(1-e**2))
    temp = beta*np.sin(M)/(1-beta*np.cos(M))
    f = M + 2*np.arctan(temp)
    return f

    
# Handling our data which are the parameters for the simulation. 
exoNames = ['b', 'c', 'd', 'e', 'f']
df = pd.read_csv('kep33Data(Sheet1).csv', names=exoNames)

# Getting the required data for inputting into the parameters.
getData = lambda idx: np.array(df.iloc[idx], dtype=float)
eArr = getData(1)
fCol = fCalc(getData(4), getData(0), eArr) # true anomaly gives us our initial position but this must be calculated from our timings.


### Orbit functions

In [6]:
def transit(x, y, dt):
    ' Define the transit to be at y=0 in the axis on the right (x positive) '
    t_transit = []
    
    if np.where(y == 0)[0] != []: # Verifying if there is a y = 0 solution
        for i in range(len(y)):
            if np.abs(x[i]) == x[i]:  # Verifying if x is positive
                # Adding time by taking the index (number of iterations) * change in t
                t_transit.append(dt*i)
    else:
        index = np.where(np.diff(np.sign(y)))[0] # Getting array where y value changes sign
        for i in index:
            if np.abs(x[i]) == x[i]: # Verifying if x is positive
                #Taking proportion to interpolate time value at y=0
                diff = np.abs(y[i]-y[i+1])
                prop = np.abs(y[i])/diff
                new_dt = prop*dt
                t = dt*(i)+new_dt #Time at index and additional time
                t_transit.append(t)
                
    return t_transit

In [7]:
def orbit_setup(params):
    mass_earth = 3e-6 # Mass earth in solar masses
    
    kep33 = rebound.Simulation() # This sets up our system
    # kep33.integrator = 'whfast'
    # Our star is at the centre of the system with no velocity etc, so no orbital elements for it except mass.
    kep33.add(m=1.29) 
    # Populating the simulation with our planets right now. 
    for i, colName in enumerate(exoNames):
        if i == 3:
            kep33.add(m=params[0], P=getData(0)[i], e=params[2], f=params[1], inc=np.pi/2)
        else:
            kep33.add(m=(getData(3)* (ast.M_earth.value/ast.M_sun.value))[i], P=getData(0)[i], e=getData(1)[i], inc=np.pi/2, f=fCol[i])

    kep33.move_to_com() # Moving our system to the centre of mass frame, which should be in the star
            

    times = np.arange(0, 365, 0.01) # Randomly spaced observations for a year
    dt = np.mean(np.gradient(times)) # Time step
    N = len(times) # Number of iterations
    # Set up 0 arrays to update with x and y values
    p1,p2,p3,p4,p5 = np.zeros((N,2)), np.zeros((N,2)),np.zeros((N,2)),np.zeros((N,2)),np.zeros((N,2))

    for i, t in enumerate(times):
        kep33.integrate(times[i]) # Integrator
        # Uodating x and y values for each planet
        p1[i] = np.array([kep33.particles[1].x, kep33.particles[1].y])
        p2[i] = np.array([kep33.particles[2].x, kep33.particles[2].y])
        p3[i] = np.array([kep33.particles[3].x, kep33.particles[3].y])
        p4[i] = np.array([kep33.particles[4].x, kep33.particles[4].y])
        p5[i] = np.array([kep33.particles[5].x, kep33.particles[5].y])
        
#     Finding transits with our planet
    t_transit_2 = transit(p2[:, 0], p2[:, 1], dt)

#     t_transit_1 = transit(p1[:, 0], p1[:, 1], dt)
#     t_transit_3 = transit(p3[:, 0], p3[:, 1], dt)
#     t_transit_4 = transit(p4[:, 0], p4[:, 1], dt)
#     t_transit_5 = transit(p5[:, 0], p5[:, 1], dt)
    
    
    return t_transit_2 #np.array[t_transit_1, t_transit_2, t_transit_3, t_transit_4, t_transit_5]

### MCMC Functions

In [8]:
def model(theta, x=x):
    # Parameters
    M, f, e = theta
    params = np.array([M, f, e])
    model = orbit_setup(params) # Getting our model transit time
    return model

def lnlike(theta, x, y, yerr):
    ymodel = model(theta)
    
    # Getting the same number of transits 
    # Maybe append zeros intead??? -------------------------
    if len(ymodel)>len(y):
        ymodel = ymodel[0:len(y)]
        
    elif len(ymodel)<len(y):
        y = y[0:len(ymodel)]
        yerr = yerr[0:len(ymodel)]

    # Getting proportionality value
    LnLike = -0.5*np.sum(((y - ymodel)/yerr)**2)
    # Getting chi square value
    chi = np.sum(((y - ymodel)**2)/ymodel)
    
    # Updating txt file
    file = open('chi_param_chain.txt', 'a')
    file.write(str(chi)+'\t'+str(theta)+'\n')
    file.close()
    
    return LnLike

def lnprob(theta, x, y, yerr):
    # Calculating probability
    like = lnlike(theta, x, y, yerr)
    return like


In [9]:
#Resetting file
reset = open("chi_param_chain.txt",'w')
reset.close()


# Adding title
file =  open('chi_param_chain.txt', 'a')
file.write('chi\tparam\n')
file.close()

# Initial params
M = (getData(3)* (ast.M_earth.value/ast.M_sun.value))[3]+1.5e-6
f = fCol[3]+0.009
e = getData(1)[3]+0.01

p_initial = np.array([M, f, e])
ndim = len(p_initial)
nwalkers = 13
niter = 300
data = (x, final_y[3][1:], y_err_array[3][1:])
p0 = [np.array(p_initial) + 1e-4 * np.random.randn(ndim) for i in range(nwalkers)]


In [10]:
def main(p0, nwalkers, niter, ndim, lnprob, data):
    sampler = emcee.EnsembleSampler(nwalkers, ndim, lnprob, args=data)

    print("Running burn-in...")
    p0, _, _ = sampler.run_mcmc(p0, 100)
    sampler.reset()

    print("Running production...")
    pos, prob, state = sampler.run_mcmc(p0, niter)

    return sampler, pos, prob, state

In [ ]:
sampler, pos, prob, state = main(p0, nwalkers, niter, ndim, lnprob, data)

Running burn-in...


/var/folders/ff/_r7359n921zcpcgllv4znt_40000gn/T/ipykernel_20769/3167903028.py:5: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if np.where(y == 0)[0] != []: # Verifying if there is a y = 0 solution
